# Jemné doladenie modelov Open AI

Tento notebook vychádza z aktuálnych odporúčaní uvedených v dokumentácii [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) od Open AI.

Jemné doladenie zlepšuje výkon základných modelov pre vašu aplikáciu tým, že ich pretrénuje s ďalšími údajmi a kontextom, ktoré sú relevantné pre konkrétny prípad použitia alebo scenár. Upozorňujeme, že techniky navrhovania promptov ako _few shot learning_ a _retrieval augmented generation_ vám umožňujú rozšíriť predvolený prompt o relevantné údaje na zlepšenie kvality. Tieto prístupy sú však obmedzené maximálnou veľkosťou tokenového okna cieľového základného modelu.

Pri jemnom doladení v podstate pretrénovávame samotný model s požadovanými údajmi (čo nám umožňuje použiť oveľa viac príkladov, než by sa zmestilo do maximálneho tokenového okna) – a nasadzujeme _vlastnú_ verziu modelu, ktorá už nepotrebuje mať príklady zadané pri inferencii. To nielenže zlepšuje efektivitu návrhu promptov (máme väčšiu flexibilitu pri využití tokenového okna na iné účely), ale potenciálne aj znižuje naše náklady (tým, že znižujeme počet tokenov, ktoré musíme modelu posielať pri inferencii).

Jemné doladenie má 4 kroky:
1. Pripravte trénovacie dáta a nahrajte ich.
1. Spustite trénovaciu úlohu na získanie jemne doladeného modelu.
1. Vyhodnoťte jemne doladený model a dolaďte ho pre lepšiu kvalitu.
1. Nasadzujte jemne doladený model na inferenciu, keď ste spokojní.

Upozorňujeme, že nie všetky základné modely podporujú jemné doladenie – [skontrolujte dokumentáciu OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) pre najnovšie informácie. Jemne doladiť môžete aj model, ktorý už bol predtým jemne doladený. V tomto návode použijeme ako cieľový základný model na jemné doladenie `gpt-35-turbo`.

---


### Krok 1.1: Pripravte si svoj dataset

Poďme si vytvoriť chatbot, ktorý vám pomôže pochopiť periodickú tabuľku prvkov tým, že bude odpovedať na otázky o prvku v podobe limeriku. V _tomto_ jednoduchom návode si len vytvoríme dataset na trénovanie modelu s niekoľkými ukážkovými príkladmi odpovedí, ktoré ukazujú očakávaný formát dát. V reálnom použití by ste potrebovali dataset s oveľa väčším počtom príkladov. Môžete tiež využiť otvorený dataset (pre vašu aplikačnú oblasť), ak existuje, a upraviť ho na použitie pri doladení modelu.

Keďže sa zameriavame na `gpt-35-turbo` a chceme jednorazovú odpoveď (chat completion), môžeme vytvoriť príklady podľa [tohto odporúčaného formátu](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), ktorý zodpovedá požiadavkám OpenAI na chat completion. Ak očakávate viacotáčkový konverzačný obsah, použili by ste [formát pre viacotáčkové príklady](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), ktorý obsahuje parameter `weight` na určenie, ktoré správy sa majú (alebo nemajú) použiť pri doladení.

V tomto návode použijeme jednoduchší formát pre jednorazovú odpoveď. Dáta sú vo formáte [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), kde je každý záznam na samostatnom riadku a predstavuje ho objekt vo formáte JSON. Nižšie je ukážka 2 záznamov – kompletnú vzorku (10 príkladov), ktorú použijeme v našom návode na doladenie, nájdete v súbore [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl). **Poznámka:** Každý záznam _musí_ byť definovaný v jednom riadku (nie rozdelený na viac riadkov ako je to bežné vo formátovanom JSON súbore)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

V reálnom použití budete potrebovať oveľa väčšiu sadu príkladov pre dobré výsledky – je to kompromis medzi kvalitou odpovedí a časom/nákladmi na doladenie. My používame malú sadu, aby sme mohli proces doladenia rýchlo ukázať. Pozrite si [tento príklad z OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) pre zložitejší návod na doladenie.


### Krok 1.2 Nahrajte svoj dataset

Nahrajte dáta pomocou Files API [ako je popísané tu](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Upozorňujeme, že na spustenie tohto kódu musíte najprv vykonať tieto kroky:
 - Nainštalovali ste Python balík `openai` (uistite sa, že používate verziu >=0.28.0 kvôli najnovším funkciám)
 - Nastavili ste premennú prostredia `OPENAI_API_KEY` na váš OpenAI API kľúč
Viac informácií nájdete v [príručke na nastavenie](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), ktorá je súčasťou kurzu.

Teraz spustite kód na vytvorenie súboru na nahranie z vášho lokálneho JSONL súboru.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Krok 2.1: Vytvorenie úlohy na doladenie pomocou SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Krok 2.2: Skontrolujte stav úlohy

Tu je niekoľko vecí, ktoré môžete robiť pomocou API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Zobrazí posledných n úloh jemného doladenia
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Získajte podrobnosti o konkrétnej úlohe jemného doladenia
- `client.fine_tuning.jobs.cancel(<job_id>)` - Zrušte úlohu jemného doladenia
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Zobrazí až n udalostí z úlohy
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Prvým krokom tohto procesu je _overenie tréningového súboru_, aby ste sa uistili, že údaje majú správny formát.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Krok 2.3: Sledovanie udalostí na monitorovanie pokroku


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Krok 2.4: Zobraziť stav v OpenAI Dashboard


Stav môžete tiež skontrolovať na webovej stránke OpenAI v sekcii _Fine-tuning_ na platforme. Tu uvidíte stav aktuálnej úlohy a môžete sledovať aj históriu predchádzajúcich spustení. Na tomto obrázku vidíte, že predchádzajúce spustenie zlyhalo a druhé bolo úspešné. Pre lepšie pochopenie: toto sa stalo, keď prvé spustenie použilo JSON súbor so zle naformátovanými záznamami – po oprave druhé spustenie prebehlo úspešne a model bol sprístupnený na použitie.

![Stav úlohy fine-tuningu](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.sk.png)


Môžete si tiež pozrieť stavové správy a metriky posúvaním nižšie vo vizuálnom paneli, ako je znázornené:

| Správy | Metriky |
|:---|:---|
| ![Správy](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.sk.png) |  ![Metriky](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.sk.png)|


### Krok 3.1: Získajte ID a otestujte jemne doladený model v kóde


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Krok 3.2: Načítanie a testovanie doladeného modelu v Playground

Doladený model môžete teraz otestovať dvoma spôsobmi. Prvým je navštíviť Playground a v rozbaľovacom zozname Models vybrať svoj novo doladený model z ponúkaných možností. Druhou možnosťou je použiť možnosť „Playground“ zobrazenú v paneli Fine-tuning (pozri snímku obrazovky vyššie), ktorá spustí _porovnávací_ pohľad, kde sa vedľa seba zobrazujú základný a doladený model pre rýchle vyhodnotenie.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.sk.png)

Stačí vyplniť systémový kontext použitý vo vašich tréningových dátach a zadať testovaciu otázku. Všimnete si, že na oboch stranách sa aktualizuje rovnaký kontext aj otázka. Spustite porovnanie a uvidíte rozdiel v odpovediach medzi modelmi. _Všimnite si, ako doladený model generuje odpoveď vo formáte, ktorý ste zadali vo svojich príkladoch, zatiaľ čo základný model len nasleduje systémový prompt_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.sk.png)

Všimnete si tiež, že porovnanie zobrazuje počet tokenov pre každý model a čas potrebný na inferenciu. **Tento konkrétny príklad je jednoduchý a slúži len na ukážku postupu, nereprezentuje však reálnu dátovú sadu alebo situáciu**. Môžete si všimnúť, že oba príklady majú rovnaký počet tokenov (systémový kontext a používateľský prompt sú identické), pričom doladený model potrebuje na inferenciu viac času (vlastný model).

V reálnych situáciách nebudete používať takýto jednoduchý príklad, ale doladíte model na reálnych dátach (napríklad produktový katalóg pre zákaznícky servis), kde bude kvalita odpovede oveľa zreteľnejšia. V _takom_ prípade bude na dosiahnutie rovnakej kvality odpovede so základným modelom potrebné viac prispôsobovať prompt, čo zvýši spotrebu tokenov a pravdepodobne aj čas spracovania na inferenciu. _Ak si to chcete vyskúšať, pozrite si príklady doladenia v OpenAI Cookbook._



---

**Vyhlásenie o vylúčení zodpovednosti**:  
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Za autoritatívny zdroj by sa mal považovať pôvodný dokument v jeho natívnom jazyku. Pre kritické informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
